# **Quora Insincere Questions Classification**

1. Mô tả bài toán
  - Kaggle có cung cấp một số công cụ Embeddings. Do đó em sẽ sử dụng các tool Embeddings kết hợp với hình của mình .

  - Ta thấy đây là một bài toán phân loại nhị phân. Theo như yêu cầu bài toán, nếu câu hỏi đó là insincere question thì label = 1, còn nếu nó là sincere question thì có  label = 0.
  
  Như vậy, từ bài toán này, ta thấy được:
  - Đầu vào: một câu hỏi trên Quora.
  - Đầu ra: kết quả của bài toán để xem câu hỏi có phải sincere hay không, nó có giá trị là 0 hoặc 1.
  

Đầu tiên, import các thư viện và print ra địa chỉ của các directory

In [ ]:
# Import các thư viện cần thiết
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
color = sns.color_palette()
%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, LSTM, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

  Sau đó, ta sẽ cho đọc các file csv (train.csv và test.csv) bằng read_csv và tạo các dataframe (trainData và testData), sau đó in ra shape của chúng.

In [ ]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

 Như vậy, ta có thể thấy được,có tổng cộng 1306122 câu hỏi trong tập train và tập test có 375806 câu hỏi. Tiếp theo, em sẽ tải thử các dataframe trainData và testData

In [ ]:
train_df

Trong trainData:
- qid là mã câu hỏi
- question_text là nội dung câu hỏi
- target là chính là label

Sau đó, em sẽ cho in số lần label 0 và label 1 tương ứng với câu hỏi sincere và insincere
xuất hiện ở tập train bằng hàm np.bincount

In [ ]:
a,b = np.bincount(train_df['target'])
print("Number of sincere question: ",a)
print("Number of insincere question: ",b)

In [ ]:
cnt_srs = train_df['target'].value_counts()

## target distribution ##
labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Target distribution',
    font=dict(size=18),
    width=400,
    height=600,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")

- Nhìn vào biểu đồ ta có thể thấy 2 nhãn không những không đồng đều mà còn có chênh lệch lớn
- Sincere question(93,8%) chiếm gấp nhiều lần insincere question(6,19%)
- Vấn đề đặt ra: Dữ liệu đang có sự chênh lệch lớn. 
- Cách giải quyết: Đánh giá mô hình bằng F1-score thay vì Accuracy

In [ ]:
test_df

Trong testData:
- qid là mã câu hỏi
- question_text là nội dung câu hỏi

**2. Preprocessing**

 Từ dữ liệu ở tập train và tập test, chỉ dùng mỗi câu hỏi không thì máy không thể hiểu được. Vì vậy, em phải biến đổi các câu hỏi trong train set và test set thành các ký tự mà máy có thể hiểu được.
 
- Đầu tiên em sẽ chia tập train thành 2 phần là train và validate
- Sau đó em sẽ đặt một số biến để dùng lúc sau, trong đó: 
    
    max_len sẽ là số lượng các từ tối đa có thể có trong một câu. Thông thường thì các câu có độ dài rất khác nhau, câu  thì quá dài, câu thì rất ngắn, vì vậy việc đặt max_len sẽ thống nhất các câu có cùng một độ dài nhất định
    
    embed_size sẽ là độ lớn của các vector mà đại diện cho mỗi từ vựng có trong các câu hỏi

In [ ]:
# lay ra 10% train de lam validate
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2021)

# dat mot so bien 
embed_size = 300 # Độ dài của mỗi vector từ
max_features = 50000 # Số lượng từ tối đa trong từ điển sẽ sử dụng
max_len = 100 # Số lượng từ tối đa trong một câu


1. Vấn đề đặt ra: Nếu để dữ liệu là các chuỗi thì máy sẽ không hiểu được. 
2. Cách giải quyết: Em nghĩ đến việc biến đổi mỗi từ thành kí tự mà máy có thể hiểu được. 
- Hàm dưới đây là hàm clean_special_chars dùng để biến đổi các ký tự đặc biệt trong các câu. Hàm này sử dụng một dict chứa các key là các ký tự đặc biệt còn các value là các ký tự mà có thể biểu diễn được dưới dạng các vector. 
- Sau đó, đối với từng từ thì sẽ được kiểm tra, nếu nó là là ký tự đặc biệt có trong dict này thì sẽ bị đổi sang dạng value của nó. 
- Sau đó, đối với các từ bị dích với các ký tự đặc biệt thì tách các ký tự đặc biệt.


In [ ]:
def clean_special_chars(text):
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    
    punct_mapping = {
        "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", 
        "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
        'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi',
        '\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''
    }
    
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    return text

- Hàm dưới đây là hàm correct_spelling, nó có cơ chế hoạt động gần giống với hàm clean_special_chars, nó cũng có một dict riêng có key là hàng loạt các từ bị sai chính tả và value là các từ đó nhưng là đúng chính tả, sau đó nó soát trong các câu hỏi, nếu có từ nào là key trong dict này thì sẽ đổi chỗ cho value của nó.

In [ ]:
def correct_spelling(x):
    mispell_dict = {
        'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 
        'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 
        'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 
        'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 
        'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 
        'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 
        'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
        'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 
        'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 
        'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 
        'pokémon': 'pokemon'
    }
    for word in mispell_dict.keys():
        x = x.replace(word, mispell_dict[word])
    return x

Hàm dataHandling dưới đây được dùng để xử lý tập train và tập set, và nó có đầu ra là tập train, tập val, tập test, label của tập train, label của tập test và cuối cùng là word_index của tập từ vựng trong các câu hỏi của tập train. Hàm dataHandling sẽ xử lý các việc sau: 
* Lấy ra label của tập train, label của tập test
* Dùng các hàm lower, clean_special_chars và correct_spelling đã nói ở trên để biến đổi tập train, tập val và tập test
* Dùng Tokenizer để tạo word index của các từ qua fit_on_texts và biến đổi các câu thành sequence các số 
* Dùng pad_sequences để chỉnh độ dài của các sequence vừa tạo dựa vào chỉ số max_len

In [ ]:
def dataHandling(trainData, testData, maxfeatures, maxlen):
    Traindf, Valdf = train_test_split(trainData, test_size=0.1, random_state=2018)
    Train_y = Traindf['target'].values
    Val_y = Valdf['target'].values
    Train_X = Traindf['question_text'].apply(lambda x: x.lower())
    Train_X = Train_X.apply(lambda x: clean_special_chars(x))
    Train_X = Train_X.apply(lambda x: correct_spelling(x))
    Val_X = Valdf['question_text'].apply(lambda x: x.lower())
    Val_X = Val_X.apply(lambda x: clean_special_chars(x))
    Val_X = Val_X.apply(lambda x: correct_spelling(x))
    Test_X = testData['question_text'].apply(lambda x: x.lower())
    Test_X = Test_X.apply(lambda x: clean_special_chars(x))
    Test_X = Test_X.apply(lambda x: correct_spelling(x))
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(Train_X))
    Train_X = tokenizer.texts_to_sequences(Train_X)
    Val_X = tokenizer.texts_to_sequences(Val_X)
    Test_X = tokenizer.texts_to_sequences(Test_X)
    Train_X = pad_sequences(Train_X, maxlen=maxlen)
    Val_X = pad_sequences(Val_X, maxlen=maxlen)
    Test_X = pad_sequences(Test_X, maxlen=maxlen)
    return Train_X, Train_y, Test_X, Val_X, Val_y, tokenizer.word_index

- Dùng hàm dataHandling này để lấy được các giá trị nói ở trên: Train_X, Train_y, Test_X, Val_X, Val_y, WordIndex

In [ ]:
Train_X, Train_y, Test_X, Val_X, Val_y, WordIndex = dataHandling(train_df, test_df, max_features, max_len)

- Ta xét đến các file được kaggle cung cấp trong file zip nên ta unzip file embeddings.zip và sẽ in ra các file có trong file zip này

In [ ]:
from zipfile import ZipFile
file_name='/kaggle/input/quora-insincere-questions-classification/embeddings.zip'
z=ZipFile(file_name)
print(z.namelist())

Tiếp theo, em sẽ extract các file trong file zip ra để lưu các file embedding vào các biến GLOVE_FILE, PARAGRAM_FILE, WIKI_NEWS_FILE.

In [ ]:
GLOVE_FILE = z.extract('glove.840B.300d/glove.840B.300d.txt')
PARAGRAM_FILE = z.extract('paragram_300_sl999/paragram_300_sl999.txt')
WIKI_NEWS_FILE = z.extract('wiki-news-300d-1M/wiki-news-300d-1M.vec')

Hàm embeddingsIndex được dùng để convert các file embedding sang dạng index hoặc là dạng dict để có thể biểu diễn nó dưới dạng key: value, trong đó value sẽ là một array với 300 element. 
- Trong đó, hàm split sẽ tách key (các từ có trong vocabulary) với các số còn lại trong dòng. 
- Sau đó, hàm fromstring để biến các số đó thành một array, ta gọi array này là value. 
- Sau đó gán các key và value lại thành một embeddings_index.

In [ ]:
def embeddingsIndex(file):
    embeddings_index = {}
    f = open(file, encoding = 'latin')
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

- Hàm embeddingsIndexF có cơ chế hoạt động gần giống với hàm embeddingsIndex do nó cũng dựa vào việc liên tục thêm vào các phần tử có dạng (key:value) vào trong một dictionary nào đó và trả ra đầu ra chính là dictionary đó.

In [ ]:
def embeddingsIndexF(file):
    embeddings_index = {}
    f = open(file, encoding = 'latin')
    for line in f:
        if(len(line) > 100):
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

- Sau đó, em dùng hàm embeddingsIndex đã tạo để tạo ra 3 index riêng cho Glove, Paragram và Fasttext

In [ ]:
glove_index = embeddingsIndex(GLOVE_FILE)
paragram_index = embeddingsIndex(PARAGRAM_FILE)
fasttext_index = embeddingsIndexF(WIKI_NEWS_FILE)

- Trong hàm embeddingsMatrix, em sẽ gọi từng từ có trong word_index vừa được tạo từ hàm dataHandling và dùng từ vừa được gọi để tra từ đó trong một embedding index, từ đó, em sẽ có một vector tương ứng với từ đó. Sau đó, em thêm các giá trị của value này vào một ma trận gọi là embedding_matrix, theo thứ tự đúng với thứ tự của các từ trong word_index. Cuối cùng, hàm này sẽ có output chính là cái embedding_matrix.

In [ ]:
def embeddingsMatrix(word_index, embeddings_index):
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None and (embedding_matrix[i].shape == embedding_vector.shape):
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

- Sau đó, em dùng hàm embeddingsMatrix vừa tạo để tạo ra các matrix cho từng embedding một

In [ ]:
glove_matrix = embeddingsMatrix(WordIndex, glove_index)
paragram_matrix = embeddingsMatrix(WordIndex, paragram_index)
fasttext_matrix = embeddingsMatrix(WordIndex, fasttext_index)

- Sau đó em sẽ xóa các biến glove_index, paragram_index, fasttext_index, trainData, WordIndex

In [ ]:
del glove_index, paragram_index, fasttext_index, train_df, WordIndex
import gc
gc.collect()

3. Model
- Đặt vấn đề: Sự dụng mô hình
- Hướng tiếp cận: Đây là một bài toán phân loại nhị phân Một số mô hình ta có thể nghĩ đến là Logistic Regression, SVM, .. nhưng em nghĩ đây là một bài toán về chuỗi nên việc xử lý chuỗi như thế nào trước khi huấn luyện quan trọng hơn là sử dụng mô hình nào. Để kết hợp với Embeddings,em sẽ sử dụng model neural network của keras để train dữ liệu này dựa trên các embeddings_matrix vừa được tạo
- Phân tích: Trong đó, model của keras gồm có rất nhiều layer và việc chọn thứ tự các layer sẽ phụ thuộc vào người lập trình. Vì vậy, em sẽ chọn model có chứa một số layer cơ bản. Trong đó, layer không thể thiếu chính là layer embedding dùng để biến đổi các từ trong tập train (lúc này, các từ đã được biểu diễn ở dạng các array qua hàm Tokenizer của keras




In [ ]:
def myModel(maxlen, embed_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(len(embedding_matrix), embed_size, weights=[embedding_matrix], input_shape=(maxlen,)))
    model.add(Bidirectional(GRU(64, return_sequences=True)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(16, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    print(model.summary())
    return model

In [ ]:
model = myModel(max_len, embed_size, glove_matrix)

Từ summary của model ta có thể thấy:
- Input sẽ là các vector chuỗi tương ứng với các câu hỏi. 
- Một chuỗi sẽ có 100 từ tương ứng với vector 100 chiều. 
- Tầng Embedding sẽ giúp máy học được nghĩa của các từ là gì. Embedding sẽ chuyển mỗi từ thành 1 vector 1x300 thể hiện nghĩa của từ đấy. Nghĩa là một câu sẽ là một vector số 100x300. 
- Tầng Bidirection sẽ giúp máy học được nghĩa của mỗi câu dựa trên thứ tự của các từ trên mạng noron. 
- Sau đó với mỗi đặc trưng trong 128 đặc trưng, tầng global sẽ chọn ra từ có đặc trưng đó tốt nhất. Các tầng còn lại trong mô hình dùng để phân loại.

Sau khi tạo các model xong, em sẽ xóa các biến là embeddings_matrix do không còn cần đến chúng nữa.

In [ ]:
del glove_matrix, paragram_matrix, fasttext_matrix
import gc
gc.collect()

Model này khi dự đoán sẽ cho các kết quả là các số thực ở tập [0,1], vì vậy, em sẽ phải biến các kết quả về tập 0 và 1 sao cho f1 score là cao nhất bằng cách đặt ra một đại lượng là threshhold, trong đó, nếu là kết quả lớn hơn threshhold thì là 1, còn lại là 0. Tại đây, để tìm được best threshold thì em sẽ dùng hàm best thresh để kiểm tra với các threshhold chạy từ 0.1 đến 0.5 thì threshhold nào sẽ cho ra kết quả cao nhất, sau đó em sẽ cho in ra với một threshhold sẽ có một f1 score tương ứng của nó.

In [ ]:
def bestThresh(Val_y, pred_val_y): 
    best_threshhold = 0
    bestf1 = 0
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        f1 = metrics.f1_score(Val_y, (pred_val_y>thresh).astype(int))
        print("F1 score at threshold {0} is {1}".format(thresh,f1))
        if(f1 > bestf1): 
            best_threshhold = thresh
            bestf1 = f1
    return best_threshhold

Bắt đầu train với tệp train_X train_Y dùng để train. Đưa dữ liệu vào mạng neural network 2 lần. Mỗi lần đưa chia nhỏ ra thành các batch_size là 1024 câu. Dữ liệu dùng để kiểm thử là val_X và val_y.

In [ ]:
model.fit(Train_X, Train_y, batch_size=1024, epochs = 2, validation_data=(Val_X, Val_y))

In [ ]:
pred_val_y = model.predict([Val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(Val_y, (pred_val_y>thresh).astype(int))))

In [ ]:
pred_y = (model.predict([Test_X], batch_size=1024, verbose=1) > bestThresh(Val_y, pred_val_y)).astype(int)

Ta thấy F1-score trong khoảng threshold 0.26-0.4 là khá tốt. 

Cuối cùng, em sẽ cho in ra bảng csv tương ứng với kết quả của model:

In [ ]:
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_y
out_df.to_csv("submission.csv", index=False)

Dưới đây là bảng kết quả của model dùng glove

In [ ]:
out_df